# 세가지 모드 선택 GUI

In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
import tkinter as tk
from tkinter import filedialog

# Function to overlay image on specific region
def overlay(image, x, y, w, h, overlay_image):
    overlay_image_resized = cv2.resize(overlay_image, (w, h))

    alpha = overlay_image_resized[:, :, 3]
    mask_image = alpha / 255

    for c in range(0, 3):
        image[y:y+h, x:x+w, c] = (overlay_image_resized[:, :, c] * mask_image) + (image[y:y+h, x:x+w, c] * (1 - mask_image))

# Function to apply eye filters
def process_frame_with_eyes(cap, face_detection, image_right_eye_path, image_left_eye_path, video_file):
    image_right_eye = cv2.imread(image_right_eye_path, cv2.IMREAD_UNCHANGED)
    image_left_eye = cv2.imread(image_left_eye_path, cv2.IMREAD_UNCHANGED)

    mp_drawing = mp.solutions.drawing_utils
    mp_face_detection = mp.solutions.face_detection

    while cap.isOpened():  # 비디오가 열려있는 동안에만 처리합니다.
        success, frame = cap.read()
        if not success:
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame)

        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.detections:
            for detection in results.detections:
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]

                h, w, _ = frame.shape
                right_eye_x = int(right_eye.x * w)
                right_eye_y = int(right_eye.y * h)
                left_eye_x = int(left_eye.x * w)
                left_eye_y = int(left_eye.y * h)

                overlay(frame, right_eye_x - 50, right_eye_y - 150, 100, 100, image_right_eye)
                overlay(frame, left_eye_x - 50, left_eye_y - 150, 100, 100, image_left_eye)

        cv2.imshow(video_file, frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to apply overlay filters
def process_frame_with_overlay(cap, face_detection, overlay_path, video_file):
    overlay_image = cv2.imread(overlay_path, cv2.IMREAD_UNCHANGED)

    mp_drawing = mp.solutions.drawing_utils
    mp_face_detection = mp.solutions.face_detection

    while cap.isOpened():  # 비디오가 열려있는 동안에만 처리합니다.
        success, frame = cap.read()
        if not success:
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame)

        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.detections:
            for detection in results.detections:
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]

                h, w, _ = frame.shape
                eyes_x = min(int(left_eye.x * w), int(right_eye.x * w))
                eyes_y = min(int(left_eye.y * h), int(right_eye.y * h)) - 50
                eyes_w = max(int(abs(right_eye.x - left_eye.x) * w), int(abs(right_eye.y - left_eye.y) * h))
                eyes_h = max(int(abs(right_eye.x - left_eye.x) * w), int(abs(right_eye.y - left_eye.y) * h))

                overlay(frame, eyes_x - 50, eyes_y - 150, eyes_w + 100, eyes_h, overlay_image)

        cv2.imshow(video_file, frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Function to process a single frame for face filters
def process_frame(frame, face_detection, filter_path):
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            filter_img = cv2.imread(filter_path, cv2.IMREAD_UNCHANGED)
            filter_resized = cv2.resize(filter_img, (w, h))

            alpha_s = filter_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(0, 3):
                frame[y:y+h, x:x+w, c] = (alpha_s * filter_resized[:, :, c] + alpha_l * frame[y:y+h, x:x+w, c])

    return frame

def process_video(selection, video_file):
    cap = cv2.VideoCapture(video_file)
    output_file = 'processed_video.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_fps = 30
    output_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    out = cv2.VideoWriter(output_file, fourcc, output_fps, output_size)
    mp_face_detection = mp.solutions.face_detection

    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
        if selection == 1:  # 기능 1: 얼굴 필터링
            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break

                processed_frame = process_frame(frame, face_detection, 'boo.png')
                out.write(processed_frame)
                cv2.imshow('처리된 비디오', processed_frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        elif selection == 2:  # 기능 2: 눈 필터
            image_right_eye_path = 'boo_left.png'  # 오른쪽 눈 이미지 경로
            image_left_eye_path = 'boo_right.png'  # 왼쪽 눈 이미지 경로
            process_frame_with_eyes(cap, face_detection, image_right_eye_path, image_left_eye_path, video_file)
        elif selection == 3:  # 기능 3: 오버레이 필터
            overlay_path = 'boo_bujeok.png'  # 오버레이 이미지 경로
            process_frame_with_overlay(cap, face_detection, overlay_path, video_file)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# process_video 함수를 사용하도록 start_functionality 수정
def start_functionality():
    selection = int(var.get())
    video_file = filedialog.askopenfilename()
    if video_file:
        process_video(selection, video_file)



# Create GUI
root = tk.Tk()
root.title("Booface 필터")

var = tk.StringVar()

tk.Label(root, text="선택:").pack()

tk.Radiobutton(root, text="부 얼굴 필터", variable=var, value=1).pack(anchor=tk.W)
tk.Radiobutton(root, text="부 헤어삔 필터", variable=var, value=2).pack(anchor=tk.W)
tk.Radiobutton(root, text="부 부적 필터", variable=var, value=3).pack(anchor=tk.W)

process_button = tk.Button(root, text="Upload", command=start_functionality)
process_button.pack()

root.mainloop()


# 부 얼굴 필터 동영상 다운로드 GUI

In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# Function to process a single frame
def process_frame(frame, face_detection):
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            filter_img = cv2.imread('boo.png', cv2.IMREAD_UNCHANGED)
            filter_resized = cv2.resize(filter_img, (w, h))

            alpha_s = filter_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(0, 3):
                frame[y:y+h, x:x+w, c] = (alpha_s * filter_resized[:, :, c] +
                                           alpha_l * frame[y:y+h, x:x+w, c])

    return frame

# Function to process the video
def process_video():
    video_file = filedialog.askopenfilename()  # Open file dialog to choose video
    if video_file:
        cap = cv2.VideoCapture(video_file)
        mp_face_detection = mp.solutions.face_detection

        # Define the codec and create VideoWriter object
        output_file = 'processed_video.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_fps = 30  # Output video frames per second
        output_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        out = cv2.VideoWriter(output_file, fourcc, output_fps, output_size)

        with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break

                processed_frame = process_frame(frame, face_detection)
                out.write(processed_frame)  # Write processed frame to output video

                cv2.imshow('Processed Video', processed_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        cap.release()
        out.release()
        cv2.destroyAllWindows()

def download_video():
    file_path = 'processed_video.mp4'  # Change this to your processed video file path
    if os.path.exists(file_path):
        os.startfile(file_path)  # Opens the file using the default app associated with the file type
    else:
        print("File not found!")
        
# Create GUI
root = tk.Tk()
root.title("부 얼굴 필터")

# Button to process video
process_button = tk.Button(root, text="Upload", command=process_video)
process_button.pack()

download_button = tk.Button(root, text="Download", command=download_video)
download_button.pack()

root.mainloop()


# 부 헤어핀 필터 동영상 다운로드 GUI

In [ ]:
import cv2
import mediapipe as mp
import tkinter as tk
from tkinter import filedialog

def overlay(image, x, y, w, h, overlay_image):
    overlay_image_resized = cv2.resize(overlay_image, (w, h))

    alpha = overlay_image_resized[:, :, 3]
    mask_image = alpha / 255

    for c in range(0, 3):
        image[y:y+h, x:x+w, c] = (overlay_image_resized[:, :, c] * mask_image) + \
                                  (image[y:y+h, x:x+w, c] * (1 - mask_image))

def process_frame(frame, face_detection, video_out):
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.detections:
        for detection in results.detections:
            keypoints = detection.location_data.relative_keypoints
            right_eye = keypoints[0]
            left_eye = keypoints[1]

            h, w, _ = frame.shape
            right_eye = (int(right_eye.x*w) - 100, int(right_eye.y*h) - 150)
            left_eye = (int(left_eye.x*w) + 30, int(left_eye.y*h) - 150)

            overlay(frame, right_eye[0], right_eye[1], 100, 100, image_right_eye)
            overlay(frame, left_eye[0], left_eye[1], 100, 100, image_left_eye)

    video_out.write(frame)

def process_video():
    video_file = filedialog.askopenfilename()
    if video_file:
        cap = cv2.VideoCapture(video_file)
        mp_face_detection = mp.solutions.face_detection

        with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
            frame_count = 0
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            video_out = cv2.VideoWriter('processed_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break

                process_frame(frame, face_detection, video_out)
                cv2.imshow('Processed Video', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        cap.release()
        video_out.release()
        cv2.destroyAllWindows()

def download_video():
    file_path = 'processed_video.mp4'
    if os.path.exists(file_path):
        os.startfile(file_path)
    else:
        print("File not found!")

root = tk.Tk()
root.title("부 헤어핀 필터")

image_right_eye = cv2.imread('boo_right.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('boo_left.png', cv2.IMREAD_UNCHANGED)

process_button = tk.Button(root, text="Upload", command=process_video)
process_button.pack()

download_button = tk.Button(root, text="Download", command=download_video)
download_button.pack()

root.mainloop()


# 부 부적 필터 동영상 다운로드 GUI

In [ ]:
import cv2
import mediapipe as mp
import tkinter as tk
from tkinter import filedialog
import os

def overlay(image, x, y, w, h, overlay_image):
    overlay_image_resized = cv2.resize(overlay_image, (w, h))
    alpha = overlay_image_resized[:, :, 3]  
    mask_image = alpha / 255  

    for c in range(0, 3):
        image[y:y+h, x:x+w, c] = (overlay_image_resized[:, :, c] * mask_image) + \
                                  (image[y:y+h, x:x+w, c] * (1 - mask_image))

def process_frame(frame, face_detection, image_eyes):
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.detections:
        for detection in results.detections:
            keypoints = detection.location_data.relative_keypoints
            right_eye = keypoints[0]
            left_eye = keypoints[1]

            h, w, _ = frame.shape
            eyes_x = min(int(left_eye.x * w), int(right_eye.x * w))
            eyes_y = min(int(left_eye.y * h), int(right_eye.y * h)) - 50
            eyes_w = max(int(abs(right_eye.x - left_eye.x) * w), int(abs(right_eye.y - left_eye.y) * h))
            eyes_h = max(int(abs(right_eye.x - left_eye.x) * w), int(abs(right_eye.y - left_eye.y) * h))

            overlay(frame, eyes_x - 50, eyes_y - 150, eyes_w + 100, eyes_h, image_eyes)

    return frame

def process_video():
    video_file = filedialog.askopenfilename()
    if video_file:
        cap = cv2.VideoCapture(video_file)
        mp_face_detection = mp.solutions.face_detection

        with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))

            video_name = 'processed_video.mp4'
            video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break

                processed_frame = process_frame(frame, face_detection, image_eyes)
                video.write(processed_frame)
                cv2.imshow('Processed Video', processed_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()

def download_video():
    file_path = 'processed_video.mp4'
    if os.path.exists(file_path):
        os.startfile(file_path)
    else:
        print("File not found!")

root = tk.Tk()
root.title("부 부적 필터")

image_eyes = cv2.imread('BOO_bujeok.png', cv2.IMREAD_UNCHANGED)

process_button = tk.Button(root, text="Upload", command=process_video)
process_button.pack()

download_button = tk.Button(root, text="Download", command=download_video)
download_button.pack()

root.mainloop()


# 웹캠 실시간 세가지 모드 선택

In [ ]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    overlay_image_resized = cv2.resize(overlay_image, (w, h))

    alpha = overlay_image_resized[:, :, 3]  # BGRA
    mask_image = alpha / 255

    for c in range(0, 3):
        image[y:y+h, x:x+w, c] = (overlay_image_resized[:, :, c] * mask_image) + (image[y:y+h, x:x+w, c] * (1 - mask_image))

def run_webcam(code_version):
    cap = cv2.VideoCapture(0)

    if code_version == 1:
        filter_img = cv2.imread('boo.png', cv2.IMREAD_UNCHANGED)
    elif code_version == 2:
        image_right_eye = cv2.imread('boo_right.png', cv2.IMREAD_UNCHANGED)
        image_left_eye = cv2.imread('boo_left.png', cv2.IMREAD_UNCHANGED)
    elif code_version == 3:
        image_eyes = cv2.imread('BOO_bujeok.png', cv2.IMREAD_UNCHANGED)

    mp_face_detection = mp.solutions.face_detection
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(frame)
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            if results.detections:
                for detection in results.detections:
                    keypoints = detection.location_data.relative_keypoints

                    if code_version == 1:
                        bboxC = detection.location_data.relative_bounding_box
                        ih, iw, _ = frame.shape
                        x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                        overlay(frame, x, y, w, h, filter_img)
                    elif code_version == 2:
                        right_eye = keypoints[0]
                        left_eye = keypoints[1]
                        h, w, _ = frame.shape
                        right_eye = (int(right_eye.x*w) - 100, int(right_eye.y*h) - 150)
                        left_eye = (int(left_eye.x*w) + 30, int(left_eye.y*h) - 150)
                        overlay(frame, right_eye[0], right_eye[1], 100, 100, image_right_eye)
                        overlay(frame, left_eye[0], left_eye[1], 100, 100, image_left_eye)
                    elif code_version == 3:
                        right_eye = keypoints[0]
                        left_eye = keypoints[1]
                        h, w, _ = frame.shape
                        eyes_x = min(int(left_eye.x * w), int(right_eye.x * w))
                        eyes_y = min(int(left_eye.y * h), int(right_eye.y * h)) - 50
                        eyes_w = max(int(abs(right_eye.x - left_eye.x) * w), int(abs(right_eye.y - left_eye.y) * h))
                        eyes_h = max(int(abs(right_eye.x - left_eye.x) * w), int(abs(right_eye.y - left_eye.y) * h))
                        overlay(frame, eyes_x - 50, eyes_y - 100, eyes_w + 100, eyes_h, image_eyes)

            cv2.imshow("Webcam", frame)

            key = cv2.waitKey(1)
            if key == ord('p'):
                if code_version == 1:
                    cv2.imwrite('captured_photo(boo_face).jpg', frame)
                    print("사진이 캡처되었습니다. 'captured_photo(boo_face).jpg'로 저장되었습니다.")
                elif code_version == 2:
                    cv2.imwrite('captured_photo(hairpin).jpg', frame)
                    print("사진이 캡처되었습니다. 'captured_photo(hairpin).jpg'로 저장되었습니다.")
                elif code_version == 3:
                    cv2.imwrite('captured_photo(bujeok).jpg', frame)
                    print("사진이 캡처되었습니다. 'captured_photo(bujeok).jpg'로 저장되었습니다.")
            elif key == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    print("코드 버전을 선택하세요:")
    print("1. 부 얼굴 필터 적용")
    print("2. 부 헤어핀 필터 적용")
    print("3. 부 부적 필터 적용")
    
    selected_version = int(input("숫자를 입력하세요 (1, 2, 3): "))
    
    run_webcam(selected_version)